# 🎬 UVG MAX
---

## 📦 Step 1: Clone & Setup

In [ ]:
# Fresh clone every time
import os, sys
os.chdir('/content')
!rm -rf uvg-max-engine
!git clone https://github.com/shya-bubu/uvg-max-engine.git
os.chdir('uvg-max-engine')
sys.path.insert(0, '/content/uvg-max-engine')

# Install deps
!pip install -q python-dotenv requests tqdm torch google-generativeai
!pip install -q azure-cognitiveservices-speech opencv-python Pillow numpy
!pip install -q librosa pyloudnorm moviepy scikit-learn

print(f"📁 {os.getcwd()}")
print("✅ Ready!")

## 🔑 Step 2: API Keys

In [ ]:
import os

# ============================================
# 🔐 PASTE YOUR API KEYS HERE
# ============================================

os.environ["GEMINI_API_KEY"] = ""  # Required
os.environ["PEXELS_KEY"] = ""       # Required
os.environ["AZURE_SPEECH_KEY"] = "" # Optional
os.environ["AZURE_SPEECH_REGION"] = "eastus"
os.environ["UVG_MOCK_MODE"] = "true"

# ============================================

print("Gemini:", "✅" if os.environ["GEMINI_API_KEY"] else "❌ REQUIRED")
print("Pexels:", "✅" if os.environ["PEXELS_KEY"] else "❌ REQUIRED")

## 🧪 Step 3: Test Imports

In [ ]:
# Force reimport
import importlib
import uvg_core.uvg_pipeline
importlib.reload(uvg_core.uvg_pipeline)

from uvg_core.uvg_pipeline import UVGPipeline, PipelineConfig
print("✅ Imports OK!")

## 🎬 Step 4: Generate Video

In [ ]:
from uvg_core.uvg_pipeline import UVGPipeline, PipelineConfig
import os

TOPIC = "The beauty of nature"

config = PipelineConfig(
    style_pack="cinematic",
    mock_mode=os.environ.get("UVG_MOCK_MODE", "true") == "true"
)

pipeline = UVGPipeline(config)
result = pipeline.run(topic=TOPIC)

print(f"{'✅ Success' if result.success else '❌ Failed'}: {result.output_path}")
if result.errors:
    for e in result.errors:
        print(f"Error: {e}")

## 🎥 Step 5: Watch

In [ ]:
from IPython.display import Video, display
import glob

videos = glob.glob("uvg_output/**/*.mp4", recursive=True)
if videos:
    display(Video(videos[0], embed=True, width=400))
else:
    print("No video found")